In [1]:
from pathlib import Path

import pyrosm

from mapillary import download_mapillary_image_by_key, download_mapillary_object_detection_by_key
from osm import add_mapillary_key_to_network

/home/snickels/Projects/street-image-classification/venv/lib/python3.9/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


### Settings

In [2]:
input_filepath = "../data/raw/berlin-latest.osm.pbf"
min_quality_score = 4
output_dir = "../data/processed/out"

### Code (see make_dataset.py)

In [3]:
osm = pyrosm.OSM(input_filepath)

In [4]:
network = osm.get_network(
    network_type="cycling",
    extra_attributes=["surface", "smoothness"]
)

In [5]:
# Filter relevant columns
network = network[["id", "geometry", "surface", "smoothness"]]

In [6]:
# Filter only records where both surface and smoothness is set
network = network[(~network["surface"].isna()) & (~network["smoothness"].isna())]

In [7]:
# Get Mapillary keys for each street
street_mapillary_df = add_mapillary_key_to_network(network.head(10), min_quality_score)

Request URL: https://a.mapillary.com/v3/images?client_id=TmxURHBKRFFFRTc1dWZscGpLWW5VUDo1ZjYyYmFjYjY5MTA3MDNk&bbox=13.525808394391273,52.48913043665209,13.527869293873632,52.49044905555783&per_page=500&sort_by=key&min_quality_score=4
DONE
Total images: 167
Request URL: https://a.mapillary.com/v3/images?client_id=TmxURHBKRFFFRTc1dWZscGpLWW5VUDo1ZjYyYmFjYjY5MTA3MDNk&bbox=13.523868068725816,52.482789917870114,13.525887018854531,52.489174160275965&per_page=500&sort_by=key&min_quality_score=4
DONE
Total images: 270
Request URL: https://a.mapillary.com/v3/images?client_id=TmxURHBKRFFFRTc1dWZscGpLWW5VUDo1ZjYyYmFjYjY5MTA3MDNk&bbox=13.635509405757002,52.36593718240352,13.637040327791425,52.369278836433104&per_page=500&sort_by=key&min_quality_score=4
DONE
Total images: 122
Request URL: https://a.mapillary.com/v3/images?client_id=TmxURHBKRFFFRTc1dWZscGpLWW5VUDo1ZjYyYmFjYjY5MTA3MDNk&bbox=13.628494275759923,52.350349756347946,13.631994532529342,52.35217994781752&per_page=500&sort_by=key&min_quality

In [8]:
street_mapillary_df

,geometry,id,mapillary_key,smoothness,surface
2,"MULTILINESTRING ((13.52783 52.49043, 13.52639 ...",4045220.0,0BdKIR4sPdlLiabOfAmD6B,bad,sett
2,"MULTILINESTRING ((13.52783 52.49043, 13.52639 ...",4045220.0,Gv8rUSeGkmDzs0yk1OtsYj,bad,sett
2,"MULTILINESTRING ((13.52783 52.49043, 13.52639 ...",4045220.0,bIfWS4heXtCoVDddCMPWi7,bad,sett
2,"MULTILINESTRING ((13.52783 52.49043, 13.52639 ...",4045220.0,GJAfy1dIDRs3mR78ddlEh2,bad,sett
2,"MULTILINESTRING ((13.52783 52.49043, 13.52639 ...",4045220.0,nLFpoV5wWGAZWFn6kpWOTh,bad,sett
...,...,...,...,...,...
51,"MULTILINESTRING ((13.63700 52.36926, 13.63713 ...",4068154.0,4HztkqBrTqYNyLhPvuvElA,bad,cobblestone
51,"MULTILINESTRING ((13.63700 52.36926, 13.63713 ...",4068154.0,ZXBFAEEofR8vA8KUcTNNTT,bad,cobblestone
51,"MULTILINESTRING ((13.63700 52.36926, 13.63713 ...",4068154.0,J9U6Q30u2OxkZz5x9Xg3Fj,bad,cobblestone
51,"MULTILINESTRING ((13.63700 52.36926, 13.63713 ...",4068154.0,SfbKzJ-H4u2y4uZ6QHL9SA,bad,cobblestone


In [9]:
# Create output dir
Path(f"{output_dir}").mkdir(parents=True, exist_ok=True)

In [13]:
# Create sub dirs
image_dir = f"{output_dir}/images"
object_detection_dir = f"{output_dir}/object_detections"

Path(image_dir).mkdir(exist_ok=True)
Path(object_detection_dir).mkdir(exist_ok=True)

In [14]:
# Export street_mapillary_df as data.csv
street_mapillary_df.to_csv(f"{output_dir}/data.csv", index=False)

In [15]:
# Download images and object detections
for _, row in street_mapillary_df.iterrows():
    download_mapillary_image_by_key(row["mapillary_key"], download_dir=image_dir)
    download_mapillary_object_detection_by_key(row["mapillary_key"],
                                               download_dir=object_detection_dir)